## data

In [1]:
import pandas as pd
df_hr = pd.read_csv('./csv/hospital_reviews.csv')
df_hr_text = df_hr[['text']].dropna()
df_khr = pd.read_csv('./csv/Seleniums_kakao_hospital_reviews_modified.csv')
df_khr.dropna(inplace=True)

In [2]:
df_hr_text[0:500]

,text
1,원장님 진료 잘봐주십니다 경력도 오래되신걸로 알고 있고 무엇보다 친절하심다
2,원장님 경력이 오래되셨다고 지인이 추천해줘서 방문했는데요 원장님 친절하시고 진료를 ...
3,통증은 꾸준하게 관리를 해줘야 한다고 해서 몇번 방문을 했는데 여기가 제일 괜찮아서...
4,직원분들 친친절하시고 원장님 진료 잘 봐주시네요 또 방문하려고요
5,친절하세요
...,...
498,좋아요
499,친절하셔서 편안히 치료받습니다.
500,내부 시설이 좋아요
501,친절하세요


In [81]:
df_khr_extract_sec = df_khr[['text', 'positive']]

In [82]:
pip install --upgrade pandas

Note: you may need to restart the kernel to use updated packages.


In [83]:
# data = {
#     'text': ['쓰레기', '쓰레기병원', '개별로임'],
#     'Value': [0,0,0]
# }
# df_khr_extract_sec.append(data, ignore_index=True)

## tokenizing

In [84]:
!pip install python-mecab-ko

In [85]:
from mecab import MeCab
mecab = MeCab()

In [86]:
mecab.pos(' 👍 👍 👍')


[('👍', 'SY'), ('👍', 'SY'), ('👍', 'SY')]

In [87]:
import re
def regex_func(text) :
    patterns = '\\\\n'
    text_regex = re.sub(pattern=patterns,repl=' ',string=text)
    return text_regex
df_khr_extract_sec['text'] = df_khr_extract_sec['text'].apply(regex_func)
df_khr_extract_sec['text']

/tmp/ipykernel_958/2554371060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_khr_extract_sec['text'] = df_khr_extract_sec['text'].apply(regex_func)


0                                          불친절!! 전공의 복귀해!
1       Very crowded and the doctors can be very unfri...
3       여 기 진 짜 별로임 애기 발에 고름이 꽉차서 누렇고 퉁퉁부어있는데 발 보더니 아무...
5       고대 구로병원에 가시거든 다른 병원가세요 저희 아버지 일반 병실에 계시다가 중환자 ...
6       여기 서류 처리 때문에 전화 했었는데 늙은 사람이 전화 받는 태도 말투가 기본도 안...
                              ...                        
3947    진짜 진짜 친절 이렇게 친절하신 의사선생님 첨봐용 병원도 꽤 커요 2층도 있고 5층...
3948                                   원장님, 간호사님 모두 친절해요.
3949                                         처음 가봤는데 친절해요
3950    여기 간호사님들 너무 친절해요 ㅠ 저녁에 7시쯤에 갔는데 애들 환자도 진짜 많았거든...
3951                                                 최고예요
Name: text, Length: 3842, dtype: object

In [106]:
mecab.pos('주말에 예약을 하지 않고 오면 한시간 이상 기다려야함')

[('주말', 'NNG'),
 ('에', 'JKB'),
 ('예약', 'NNG'),
 ('을', 'JKO'),
 ('하', 'VV'),
 ('지', 'EC'),
 ('않', 'VX'),
 ('고', 'EC'),
 ('오', 'VV'),
 ('면', 'EC'),
 ('한', 'MM'),
 ('시간', 'NNG'),
 ('이상', 'NNG'),
 ('기다려야', 'VV+EC'),
 ('함', 'VX+ETN')]

In [107]:
def extract(series) :
    morphs_nouns_list = []
    for sentence in series:
        morphs_nouns = ''
        for morph, tag in mecab.pos(sentence) : 
            if tag in ['NNG', 'NNP', 'NNB', 'NNBC', 'NP', 'NR', 'VA', 'MAG', 'SY', 'VV+EC'] :
                morphs_nouns += (morph+' ')            
        morphs_nouns_list.append(morphs_nouns)
        pass
    return morphs_nouns_list
morphs_nouns_list = extract(df_khr_extract_sec['text'])

In [108]:
df_khr_extract_sec['text_tokenized'] = morphs_nouns_list

/tmp/ipykernel_958/4088541910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_khr_extract_sec['text_tokenized'] = morphs_nouns_list


In [145]:
df_khr_extract_sec.query('positive==0')

,text,positive,text_tokenized
0,불친절!! 전공의 복귀해!,0.0,불친절 ! 전공 복귀
1,Very crowded and the doctors can be very unfri...,0.0,
3,여 기 진 짜 별로임 애기 발에 고름이 꽉차서 누렇고 퉁퉁부어있는데 발 보더니 아무...,0.0,짜 별로 애기 발 고름 꽉 누렇 퉁퉁 발 이상 없 고름 발 꽉 눌러서 애 비명 꾹꾹...
5,고대 구로병원에 가시거든 다른 병원가세요 저희 아버지 일반 병실에 계시다가 중환자 ...,0.0,고대 구로 병원 병원 저희 아버지 일반 병실 중환자 실 때 일반 병동 간호사 한태 ...
6,여기 서류 처리 때문에 전화 했었는데 늙은 사람이 전화 받는 태도 말투가 기본도 안...,0.0,여기 서류 처리 때문 전화 사람 전화 태도 말투 기본 안 도저히 일 사람 구분 안 ...
...,...,...,...
3933,주사 참 못 놔요 본원 간호사이신 게 분명한데 이제 막 실습 나온 학생 분이 연습하...,0.0,주사 참 못 놔요 본원 간호사 분명 이제 실습 학생 분 연습 생각 ^^ 수액 혈관 ...
3934,전화도 처안받고 그냥 지들멋대로,0.0,전화 처 안 그냥 지 멋대로
3935,불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불...,0.0,불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친절 불친...
3940,2018년도 여기서 비중격 수술이랑 축농증 수술을 받았던 환자인데요~~ 처음부터 ...,0.0,년 여기 비중 격 수술 축농증 수술 환자 ~~ 처음 수술 유도 .. 끝 코 사이 천...


In [110]:
df_khr_extract_sec

,text,positive,text_tokenized
0,불친절!! 전공의 복귀해!,0.0,불친절 ! 전공 복귀
1,Very crowded and the doctors can be very unfri...,0.0,
3,여 기 진 짜 별로임 애기 발에 고름이 꽉차서 누렇고 퉁퉁부어있는데 발 보더니 아무...,0.0,짜 별로 애기 발 고름 꽉 누렇 퉁퉁 발 이상 없 고름 발 꽉 눌러서 애 비명 꾹꾹...
5,고대 구로병원에 가시거든 다른 병원가세요 저희 아버지 일반 병실에 계시다가 중환자 ...,0.0,고대 구로 병원 병원 저희 아버지 일반 병실 중환자 실 때 일반 병동 간호사 한태 ...
6,여기 서류 처리 때문에 전화 했었는데 늙은 사람이 전화 받는 태도 말투가 기본도 안...,0.0,여기 서류 처리 때문 전화 사람 전화 태도 말투 기본 안 도저히 일 사람 구분 안 ...
...,...,...,...
3947,진짜 진짜 친절 이렇게 친절하신 의사선생님 첨봐용 병원도 꽤 커요 2층도 있고 5층...,2.0,진짜 진짜 친절 이렇게 친절 의사 선생 첨 병원 꽤 층 있 층 있 선생 많 것 같
3948,"원장님, 간호사님 모두 친절해요.",2.0,원장 간호사 모두 친절
3949,처음 가봤는데 친절해요,2.0,처음 가 친절
3950,여기 간호사님들 너무 친절해요 ㅠ 저녁에 7시쯤에 갔는데 애들 환자도 진짜 많았거든...,2.0,여기 간호사 너무 친절 저녁 시 애 환자 진짜 많 너무 정신 못 계속 친절 존경 ....


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [112]:
tfidfVectorizer.fit(df_khr_extract_sec['text_tokenized'])

TfidfVectorizer(ngram_range=(1, 2))

In [146]:
positive_random_idx = df_khr_extract_sec.query('positive ==2').sample(1190, random_state=12).index.tolist()
negative_random_idx = df_khr_extract_sec.query('positive ==0').sample(1190, random_state=12).index.tolist()
random_idx = positive_random_idx + negative_random_idx 
random_idx

[1879,
 3515,
 1382,
 1878,
 1379,
 1584,
 1529,
 1807,
 3425,
 1984,
 1130,
 1452,
 2033,
 3196,
 1583,
 3901,
 729,
 559,
 2697,
 922,
 795,
 3327,
 2249,
 3115,
 174,
 715,
 2850,
 3546,
 3903,
 3287,
 1081,
 2125,
 3827,
 341,
 3698,
 3441,
 3662,
 3049,
 1685,
 1706,
 2108,
 3065,
 1677,
 3009,
 2394,
 1608,
 3427,
 1182,
 1275,
 3380,
 1738,
 3926,
 1698,
 1197,
 773,
 3293,
 3939,
 3104,
 3250,
 1461,
 602,
 801,
 3040,
 886,
 2287,
 748,
 2905,
 1816,
 2244,
 740,
 3438,
 2667,
 860,
 1354,
 2766,
 1259,
 2876,
 2305,
 2906,
 1648,
 3244,
 3422,
 1669,
 1113,
 1265,
 737,
 2682,
 3442,
 3910,
 3408,
 2245,
 547,
 3152,
 2695,
 2186,
 2522,
 1798,
 625,
 1542,
 1931,
 2674,
 2393,
 3833,
 1430,
 2962,
 538,
 728,
 3218,
 466,
 2879,
 3718,
 353,
 587,
 1442,
 767,
 3703,
 2612,
 3524,
 342,
 2122,
 3449,
 2012,
 3484,
 3426,
 2590,
 410,
 2714,
 516,
 1590,
 1715,
 3676,
 2921,
 1540,
 863,
 3445,
 1601,
 3103,
 3641,
 1152,
 746,
 3846,
 2366,
 3656,
 1219,
 3391,
 2013,
 1624,

In [147]:
features = df_khr_extract_sec.loc[random_idx]['text_tokenized']
target = df_khr_extract_sec.loc[random_idx]['positive']

In [148]:
tokenized_features = tfidfVectorizer.transform(features)

In [149]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_jobs=-1)

In [150]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(tokenized_features, target, test_size=0.1, random_state=55)
features_train.shape, features_test.shape, target_train.shape, target_test.shape

((2142, 37134), (238, 37134), (2142,), (238,))

In [151]:
model.fit(features_train,target_train)

RandomForestClassifier(n_jobs=-1)

In [152]:
model.predict(features_test)

array([2., 2., 2., 0., 2., 0., 0., 2., 0., 2., 0., 0., 0., 0., 0., 2., 2.,
       2., 2., 2., 0., 2., 2., 0., 0., 2., 2., 0., 2., 0., 2., 0., 0., 2.,
       0., 2., 2., 0., 2., 0., 2., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0.,
       0., 2., 0., 0., 0., 2., 2., 0., 2., 0., 2., 2., 2., 2., 0., 2., 0.,
       2., 2., 0., 0., 2., 0., 0., 2., 0., 2., 2., 2., 2., 0., 2., 0., 2.,
       0., 0., 0., 0., 0., 0., 2., 0., 2., 2., 0., 2., 2., 2., 2., 2., 0.,
       2., 2., 2., 0., 2., 2., 2., 0., 0., 0., 2., 2., 2., 0., 0., 2., 0.,
       2., 0., 0., 0., 2., 2., 0., 0., 2., 2., 2., 2., 0., 2., 0., 0., 0.,
       2., 2., 2., 0., 2., 2., 0., 2., 0., 2., 0., 2., 0., 0., 0., 0., 2.,
       0., 2., 2., 0., 2., 2., 0., 0., 2., 0., 0., 2., 0., 2., 0., 2., 0.,
       2., 0., 2., 2., 2., 2., 2., 0., 0., 2., 0., 2., 2., 2., 2., 0., 0.,
       2., 2., 2., 2., 2., 0., 0., 2., 2., 0., 0., 0., 0., 0., 0., 2., 2.,
       0., 2., 2., 2., 2., 0., 0., 2., 2., 0., 2., 2., 0., 2., 0., 0., 2.,
       2., 2., 2., 0., 2.

In [153]:
from sklearn.metrics import classification_report

In [154]:
target_predict = model.predict(features_test)
print(classification_report(target_test, target_predict))

              precision    recall  f1-score   support

         0.0       0.91      0.89      0.90       114
         2.0       0.90      0.92      0.91       124

    accuracy                           0.91       238
   macro avg       0.91      0.91      0.91       238
weighted avg       0.91      0.91      0.91       238



In [155]:
# make predict function
def sample_predict(features):
    tokenized_features = tfidfVectorizer.transform(extract(features))
    return model.predict(tokenized_features)
sample_predict(['그냥 그래요'])

array([2.])

In [163]:
text=sample_predict(['돌팔이 의사들 ㅉㅉ'])
if text[0] == 0 :
    print('부정')
elif text[0] == 2 :
    print('긍정')


부정


In [165]:
# 재사용 위해 model을 파일로 저장
import pickle # 메모리 인스턴스 클래스를 이진파일로 저장

In [166]:
with open('Hospital_reviews_positive.pkl', 'wb') as pickle_file:
  pickle.dump(obj=model, file=pickle_file)

In [167]:
with open('Hospital_reviews_vectorize.pkl', 'wb') as pickle_file:
  pickle.dump(obj=tfidfVectorizer, file=pickle_file)